<center><h1>Exploring European Union (EU) Artificial Intelligence Act (AI Act) with Gemma</h1></center>

<center><img src="https://cdn.analyticsvidhya.com/wp-content/uploads/2024/02/image-488.png" width="600"></center>
# Introduction


We will build a Retrieval Augmented Generation (RAG) system based on Gemma, to query and get contextual answers about the European Union Artificial Intelligence Act (AI Act).


## What is (EU) AI Act?

The Artificial Intelligence Act (AI Act) is a European Union regulation on artificial intelligence (AI) in the European Union. Proposed by the European Commission on 21 April 2021 and passed on 13 March 2024, it aims to establish a common regulatory and legal framework for AI (from Artificial Intelligence Act page, Wikipedia).

## What is Gemma?

Gemma is a collection of lightweight source generative AI models designed to be used mostly by developers and researchers. Created by Google DeepMind research lab that also developed Gemini, Gemma is available in several versions, with 2B and 7B parameters.

## What is RAG?

Retriever augmented generation (RAG) is a system that improves the response generated by a LLM in two ways:

* (1) The information is retrieved from a dataset that is stored in vector database; the query is used to perform similarity search in the documents stored in the vector database.
* (2) By restraining the context provided to the LLM to content that is similar with the initial query, stored in the vector database, we can reduce significantly (or even eliminate) LLM's halucinations, since the answer is provided from the context of the stored documents.

An important advantage of this approach is that we do not need to fine-tune the LLM with our custom data; instead, the data is ingested (cleaned, transformed, chunked, and indexed in the vector database).

## How we will proceed?

We create two classes:

* AIAgent - An AI Agent that query Gemma LLM using a custom prompt that instruct Gemma to generate and answer (from the query) by refering to the context (as well provided); the answer to the AI Agent query function is then returned.

* RAGSystem - initialized with the dataset with Data Science information, with an AIAgent object. In the init function of this class, we ingest the data from the dataset in the vector database. This class have as well a query member function. In this function we first perform similarity search with the query to the vector database. Then, we call the generate function of the ai agent object. Before returning the answer, we use a predefined template to compose the overal response from the question, answer and the context retrieved.



# Prerequisites: packages instalation and configurations


In [1]:
# install required libraries
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install accelerate
!pip install -i https://pypi.org/simple/ bitsandbytes
!pip install langchain
!pip install sentence-transformers
!pip install chromadb

Looking in indexes: https://pypi.org/simple/
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 MB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 812.8/812.8 kB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 52.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 279.2/279.2 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.9/86.9 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.8/144.8 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 3.2 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 21.3
    Uninstalling packaging-21.3:
      Successfully uninstalled packaging-21.3
  Attempting uninstall: orjson
    Found existing installation: orjson 3.9.10
    Uninstalling orjson-3.9.10:
      Successfully uninstalled orjson-3.9.10
ERR

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM

from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

from IPython.display import display, Markdown

# Setup the RAG system

We initialize an AIAgent object and then a RAGSystem object. 
The initialization of the RAGSystem object requires an AIAgent object (we pass it in the initialization, as a parameter).
In the init of AIAgent we set the generation part of the RAG system.

In the init of RAGSystem, the following operations are done:
* Conversion of pdf document(s) to text;
* Chunk the document(s) with partial superposition of the chunks;
* Use embeddings to encode the document chunks in the vector database;
* Create the vector storage as a persistent Chroma database;
* Set Chroma as a retriever.
All these steps are necesary to define the retriever part of the RAG system.

We will set the system so that, in the response, we pack not only the initial question and the answer, but also the context from which the answer was extracted.

## Define the AI Agent

In [3]:
class AIAgent:
    """This is Gemma 2b-it based assistant that replies given the retrieved documents"""
    def __init__(self):
        self.tokenizer = AutoTokenizer.from_pretrained("/kaggle/input/gemma/transformers/2b-it/2")
        self.gemma_lm = AutoModelForCausalLM.from_pretrained("/kaggle/input/gemma/transformers/2b-it/2")

    def create_prompt(self, query, context):
        # prompt template
        prompt = f"""
        You are an AI Agent specialized to answer to questions about European Union
        Artificial Inteligence Act (AI Act).
        Explain the concepts or answer the questions about AI Act.
        In order to create the answer, please only use the information from the
        context provided (Context). Do not include other information.
        Answer with simple words.
        If needed, include also explanations.
        Question: {query}
        Context: {context}
        Answer:
        """
        return prompt
    
    def generate(self, query, retrieved_info):
        prompt = self.create_prompt(query, retrieved_info)
        input_ids = self.tokenizer(query, return_tensors="pt").input_ids
        # Answer generation
        answer = self.gemma_lm.generate(
            input_ids,
            max_length=512, # limit the answer to 512
        )
        # Decode and return the answer
        answer = self.tokenizer.decode(answer[0], skip_special_tokens=True)
        return answer

# Define the RAG system

In [4]:
class RAGSystem:
    """Sentence embedding based Retrieval Based Augmented generation.
        Given pdf file, retriever finds a number of num_retrieved_docs
        relevant document chunks"""
    def __init__(self, ai_agent, num_retrieved_docs=3):
        # ingest the document data
        # A pdf file contains the AI Act text 
        self.num_docs = num_retrieved_docs
        self.ai_agent = ai_agent
        loader = PyPDFLoader("//kaggle/input/eu-ai-act-complete-text/aiact_final_draft.pdf")
        documents = loader.load()
        self.template = "\n\nQuestion:\n{question}\n\nAnswer:\n{answer}\n\nContext:\n{context}"
        # creates chunks, division of the document(s) in multiple,
        # partially superposed chunks
        # by partially superposing the chunks, we ensure we will 
        # find the right context upon querying the vector db
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=800, 
            chunk_overlap=100)
        all_splits = text_splitter.split_documents(documents)
        # create a vectorstore database
        # create the embeddings
        embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
        # create the Chroma vector database (persistent)
        self.vector_db = Chroma.from_documents(documents=all_splits, 
                                               embedding=embeddings, 
                                               persist_directory="chroma_db")
        # set the vector db as a retriever
        self.retriever = self.vector_db.as_retriever()

    def retrieve(self, query):
        # retrieve top k similar documents to query
        docs = self.retriever.get_relevant_documents(query)
        return docs
    
    def query(self, query):
        # generate the answer
        context = self.retrieve(query)
        answer = self.ai_agent.generate(query, context)
        
        return self.template.format(question=query, 
                                   answer=answer,
                                   context=context)
        

## Utility function

In [5]:
def colorize_text(text):
    for word, color in zip(["Question", "Answer", "Context"], ["blue", "red", "green"]):
        text = text.replace(f"\n\n{word}:", f"\n\n**<font color='{color}'>{word}:</font>**")
    return text

# Query the RAG system about AI Act



In [6]:
ai_agent = AIAgent()
rag_system = RAGSystem(ai_agent)

Gemma's activation function should be approximate GeLU and not exact GeLU.
Changing the activation function to `gelu_pytorch_tanh`.if you want to use the legacy `gelu`, edit the `model.config` to set `hidden_activation=gelu`   instead of `hidden_act`. See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

## Test 1

In [7]:
answer = rag_system.query("How is performed the testing of high-risk AI systems in real world conditions?")
display(Markdown(colorize_text(answer)))



**<font color='blue'>Question:</font>**
How is performed the testing of high-risk AI systems in real world conditions?

**<font color='red'>Answer:</font>**
How is performed the testing of high-risk AI systems in real world conditions?

**High-risk AI systems** are those that have the potential to cause significant harm if they are not tested and validated thoroughly. This is especially true for systems that handle critical decisions or that are used in environments where human life or property is at stake.

**Testing high-risk AI systems in real world conditions** is essential for ensuring that they are safe and reliable before they are deployed. This involves a number of steps, including:

* **Identifying potential risks:** This involves identifying all of the potential risks associated with the system, such as those related to privacy, security, and safety.
* **Developing a testing plan:** This plan should outline the steps that will be taken to test the system in real world conditions, as well as the criteria that will be used to determine the success of the testing.
* **Obtaining ethical approval:** This approval is necessary before testing can begin, and it should ensure that the testing is conducted in a responsible manner.
* **Setting up a testing environment:** This environment should be designed to mimic the real world environment in as many ways as possible.
* **Testing the system:** This involves running the system in the testing environment and evaluating its performance.
* **Analyzing the results:** The results of the testing are then analyzed to determine whether the system meets the safety and reliability requirements.
* **Making necessary revisions:** If the system does not meet the requirements, the developers make necessary revisions and repeat the testing process.

**Here are some of the challenges associated with testing high-risk AI systems in real world conditions:**

* **The complexity of real world environments:** Real world environments are complex and dynamic, which can make it difficult to create a testing environment that is accurate enough to reflect the real world.
* **The lack of access to real world data:** In some cases, it may not be possible to access real world data that is needed to test the system.
* **The need to involve stakeholders:** Testing high-risk AI systems often involves involving a number of stakeholders, including developers, testers, and users. This can make it difficult to coordinate the testing process and ensure that all stakeholders are involved in the decision-making process.

Despite these challenges, testing high-risk AI systems in real world conditions is essential for ensuring that they are safe and reliable before they are deployed. By following the steps outlined above, developers can increase the likelihood that high-risk AI systems will

**<font color='green'>Context:</font>**
[Document(page_content='5662/24\n \n \n \nRB/ek\n \n183\n \n \nTREE.2.B\n \nLIMITE\n \nEN\n \n \nArticle 54a\n \nTesting of high\n-\nrisk AI systems in real wo\nrld conditions outside AI regulatory sandboxes\n \n1.\n \nTesting of AI systems in real world conditions outside AI regulatory sandboxes may be \nconducted by providers or prospective providers of high\n-\nrisk AI systems listed in Annex \nIII, in accordance with the prov\nisions of this Article and the real\n-\nworld testing plan \nreferred to in this Article, without prejudice to the prohibitions under Article 5.\n \n \nThe detailed elements of the real world testing plan shall be specified in implementing acts \nadopted by the Commissi\non in accordance with the examination procedure referred to in \nArticle 74(2).', metadata={'page': 182, 'source': '//kaggle/input/eu-ai-act-complete-text/aiact_final_draft.pdf'}), Document(page_content='on in accordance with the examination procedure referred to in \nArticle 74(2).\n \n \n \nThis provision shall be without prejudice to Union or national law for the testing in real \nworld conditions of high\n-\nrisk AI systems related to products covered by legislation l\nisted \nin Annex II.\n \n2.\n \nProviders or prospective providers may conduct testing of high\n-\nrisk AI systems referred to \nin Annex III in real world conditions at any time before the placing on the market or \nputting into service of the AI system on their own or in \npartnership with one or more \nprospective deployers. \n \n3.\n \nThe testing of high\n-\nrisk AI systems in real world conditions under this Article shall be \nwithout prejudice to ethical review that may be required by national or Union law.\n \n4.\n \nProviders or prospective', metadata={'page': 182, 'source': '//kaggle/input/eu-ai-act-complete-text/aiact_final_draft.pdf'}), Document(page_content='shall give consideration to whether in view of its intended purpose the high\n-\nrisk AI system', metadata={'page': 116, 'source': '//kaggle/input/eu-ai-act-complete-text/aiact_final_draft.pdf'}), Document(page_content='well as to gain awareness about the opportunities and risks of AI and possible harm it \ncan cause; \n \n(bi)\n \n‘testing in real world conditio\nns’ means the temporary testing of an AI system for its \nintended purpose in real world conditions outside of a laboratory or otherwise \nsimulated environment with a view to gathering reliable and robust data and to \nassessing and verifying the conformity of \nthe AI system with the requirements of this \nRegulation; testing in real world conditions shall not be considered as placing the AI \nsystem on the market or putting it into service within the meaning of this Regulation, \nprovided that all conditions under Art\nicle 53 or Article 54a are fulfilled;\n \n(bj)\n \n‘subject’ for the purpose of real world testing means a natural person who', metadata={'page': 102, 'source': '//kaggle/input/eu-ai-act-complete-text/aiact_final_draft.pdf'})]

## Test 2

In [8]:
answer = rag_system.query("What are the operational obligations of notified bodies?")
display(Markdown(colorize_text(answer)))



**<font color='blue'>Question:</font>**
What are the operational obligations of notified bodies?

**<font color='red'>Answer:</font>**
What are the operational obligations of notified bodies?

**Operational obligations of notified bodies:**

* To comply with the applicable requirements of the relevant harmonized standards and regulations.
* To maintain impartiality and objectivity in their decision-making processes.
* To provide accurate and timely reports on their activities and findings.
* To cooperate with the relevant authorities and stakeholders.
* To document and maintain a comprehensive quality management system.
* To conduct internal audits and external audits to ensure compliance with the requirements.
* To participate in the relevant harmonized standards and regulations review bodies.
* To provide training and support to the relevant personnel.
* To maintain a documented record of all activities and findings.
* To issue certificates and reports to the relevant authorities and stakeholders.
* To ensure the quality of their services and products.
* To comply with the ethical principles and good governance practices.

**<font color='green'>Context:</font>**
[Document(page_content='5662/24\n \n \n \nRB/ek\n \n145\n \n \nTREE.2.B\n \nLIMITE\n \nEN\n \n \nArticle 33\n \nRequirements relating t\no notified bodies \n \n1.\n \nA notified body shall be established under national law of a Member State and have legal \npersonality.\n \n2.\n \nNotified bodies shall satisfy the organisational, quality management, resources and process \nrequirements that are necessary to fu\nlfil their tasks, as well as suitable cybersecurity \nrequirements.\n \n3.\n \nThe organisational structure, allocation of responsibilities, reporting lines and operation of \nnotified bodies shall be such as to ensure that there is confidence in the performance by \nan\nd in the results of the conformity assessment activities that the notified bodies conduct.\n \n4.\n \nNotified bodies shall be independent of the provider of a high\n-', metadata={'page': 144, 'source': '//kaggle/input/eu-ai-act-complete-text/aiact_final_draft.pdf'}), Document(page_content="Coordination of notified bodies\n \n1.\n \nThe Commission shall ensure that, with regard to high\n-\nrisk AI systems, appropriate \ncoordination and cooperation \nbetween notified bodies active in the conformity assessment \nprocedures pursuant to this Regulation are put in place and properly operated in the form \nof a sectoral group of notified bodies.\n \n2.\n \nThe notifying authority shall ensure that the bodies notified b\ny them participate in the work \nof that group, directly or by means of designated representatives.\n \n2a.\n \nThe Commission shall provide for the exchange of knowledge and best practices between \nthe Member States' notifying authorities. \n \nArticle 39\n \nConformity ass\nessment bodies of third countries", metadata={'page': 151, 'source': '//kaggle/input/eu-ai-act-complete-text/aiact_final_draft.pdf'}), Document(page_content='their behalf and under their responsibility.\n \n10.\n \nNotified bodies shall have sufficient\n \ninternal competences to be able to effectively evaluate \nthe tasks conducted by external parties on their behalf. The notified body shall have \npermanent availability of sufficient administrative, technical, legal and scientific personnel \nwho possess experi\nence and knowledge relating to the relevant types of artificial \nintelligence systems, data and data computing and to the requirements set out in Chapter 2 \nof this Title.\n \n11.\n \nNotified bodies shall participate in coordination activities as referred to in Art\nicle 38. They \nshall also take part directly or be represented in European standardisation organisations, or', metadata={'page': 145, 'source': '//kaggle/input/eu-ai-act-complete-text/aiact_final_draft.pdf'}), Document(page_content='Recommendation 2003/361/EC.\n \n3.\n \nNotified bodies shall make available and submit upon \nrequest all relevant documentation, \nincluding the providers’ documentation, to the notifying authority referred to in Article 30 \nto allow that authority to conduct its assessment, designation, notification, monitoring \nactivities and to facilitate the asses\nsment outlined in this Chapter.\n \nArticle 35\n \nIdentification numbers and lists of notified bodies designated under this Regulation\n \n1.\n \nThe Commission shall assign an identification number to notified bodies. It shall assign a \nsingle number, even where a body \nis notified under several Union acts.\n \n2.\n \nThe Commission shall make publicly available the list of the bodies notified under this', metadata={'page': 147, 'source': '//kaggle/input/eu-ai-act-complete-text/aiact_final_draft.pdf'})]

## Test 3

In [9]:
answer = rag_system.query("Please describe the further processing of personal data for developing certain AI systems in the public interest")
display(Markdown(colorize_text(answer)))



**<font color='blue'>Question:</font>**
Please describe the further processing of personal data for developing certain AI systems in the public interest

**<font color='red'>Answer:</font>**
Please describe the further processing of personal data for developing certain AI systems in the public interest.

Further processing of personal data for developing certain AI systems in the public interest is a complex and multifaceted process that involves several stages and considerations. Here's a breakdown of the further processing steps involved:

**1. Data collection and access:**

* The first step involves collecting and accessing the personal data required for the AI system. This may include data from various sources, such as government records, social media platforms, healthcare databases, and surveys.
* Data privacy and security measures are crucial throughout this process to protect individuals' sensitive information.

**2. Data cleaning and preparation:**

* Once the data is collected, it needs to be cleaned and prepared for use in the AI system. This involves removing duplicates, correcting errors, and transforming data into the appropriate format for the AI model.
* Data transformation may involve encoding categorical variables, scaling numerical values, and removing irrelevant data points.

**3. Data transformation and feature engineering:**

* The cleaned data is transformed into a format that is suitable for training the AI model. This may involve creating new features from existing data or combining multiple datasets.
* Feature engineering involves identifying and selecting relevant features that are likely to be predictive of the desired outcome.

**4. Data augmentation and ethical considerations:**

* To increase the size and diversity of the training data, data augmentation techniques may be employed. This involves creating new synthetic data points based on existing data, ensuring that the AI model is exposed to a wider range of examples.
* Ethical considerations are crucial during data augmentation to ensure that the new synthetic data points do not perpetuate biases or introduce irrelevant information into the model.

**5. Model training and evaluation:**

* The transformed data is used to train the AI model. This involves feeding the data into the model and adjusting its parameters to minimize errors.
* Once the model is trained, it is evaluated to assess its performance on a separate test dataset. Metrics such as accuracy, precision, and recall are used to evaluate the model's accuracy andgeneralizability.

**6. Data security and privacy:**

* Throughout the entire data processing and model development process, data security and privacy are paramount. This involves implementing robust security measures, such as encryption, access controls, and data minimization practices.

**7. Monitoring and maintenance:**

* Once the AI system is deployed, it needs to be monitored and maintained to ensure its continued accuracy and effectiveness. This involves collecting and

**<font color='green'>Context:</font>**
[Document(page_content='categories of personal data, as a matter of substantial public interest within t\nhe meaning of \nArticle 9(2)(g) of Regulation (EU) 2016/679 and Article 10(2)g) of Regulation (EU) \n2018/1725.\n \n(46)\n \nHaving comprehensible information on how high\n-\nrisk AI systems have been developed \nand how they perform throughout their lifetime is essential t\no enable traceability of those \nsystems, verify compliance with the requirements under this Regulation, as well as \nmonitoring of their operations and post market monitoring. This requires keeping records \nand the availability of a technical documentation, co\nntaining information which is \nnecessary to assess the compliance of the AI system with the relevant requirements and', metadata={'page': 46, 'source': '//kaggle/input/eu-ai-act-complete-text/aiact_final_draft.pdf'}), Document(page_content='(f)  \n \nthe records of processing activities pursuant to Regulation (EU\n) 2016/679, Directive \n(EU) 2016/680 and Regulation (EU) 2018/1725 includes justification why the \nprocessing of special categories of personal data was strictly necessary to detect and \ncorrect biases and this objective could not be achieved by processing ot\nher data.\n \n6.\n \nFor the development of high\n-\nrisk AI systems not using techniques involving the training of \nmodels, paragraphs 2 to 5 shall apply only to the testing data sets.\n \nArticle 11\n \nTechnical documentation \n \n1.\n \nThe technical documentation of a high\n-\nrisk A\nI system shall be drawn up before that system \nis placed on the market or put into service and shall be kept up\n-\nto date.', metadata={'page': 119, 'source': '//kaggle/input/eu-ai-act-complete-text/aiact_final_draft.pdf'}), Document(page_content='1.\n \nIn the AI regulatory sandbox personal data lawfully collected for other purposes may be \nprocessed solely for the purposes of developing, training and testing certain AI systems in \nthe sandbox when all of the foll\nowing conditions are met:', metadata={'page': 179, 'source': '//kaggle/input/eu-ai-act-complete-text/aiact_final_draft.pdf'}), Document(page_content='and \ngeneral purpose AI models, tools, services or processes of an AI system. Free and open\n-\nsource AI components can be provided through different channels, including their \ndevelopment on open repositories. For the purpose of this Regulation, AI components\n \nthat \nare provided against a price or otherwise monetised, including through the provision of \ntechnical support or other services, including through a software platform, related to the AI \ncomponent, or the use of personal data for reasons other than exclus\nively for improving the \nsecurity, compatibility or interoperability of the software, with the exception of \ntransactions between micro enterprises, should not benefit from the exceptions provided to', metadata={'page': 59, 'source': '//kaggle/input/eu-ai-act-complete-text/aiact_final_draft.pdf'})]

# Conclusions


We created a RAG system, using Gemma (as a LLM), Langchain (for the pdf loader), a custom prompt (for generation), and Chroma (as vector database, for retriever).  

We ingested the full text of AI Act and tested the RAG system with few questions about the AI Act.  

Besides the question and answer, the RAG system is also returning the relevant context used to answer to the question.
